# Matrix Completion, part 1

This lab is a precursor to the "big" lab we'll do next week on matrix completion.  You will build much of the infrastructure for this assignment this week.  We will use our very small foods dataset to make sure all our code runs properly.  In some ways, this dataset is perfect for development - it is small, so our code will run fast, and the matrix is quite dense - that is, there aren't many missing data points.

Begin by running the below to create your `foodMatrix` matrix you'll be completing.  Note you'll have both `foodFrame` which is the pandas dataframe and `foodMatrix` which is the numpy array.

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from copy import deepcopy

In [4]:
df = pd.read_csv('datasets/foodsAndMovies.csv')
foodFrame = df[['Broccoli', 'Mushrooms', 'Beef Tacos', 'Salads', 'Black Licorice', 'Steak', 'Grilled Chicken', 'Mayonnaise', 'Candy Corn', 'Pulled Pork', 'Spicy Mustard', 'Raw Oysters', 'Bananas', 'Avocado', 'Eggs', 'Olives', 'Tofu', 'Cottage Cheese']]
foodMatrix=foodFrame.values
print(foodMatrix)

[[ 4.  4.  5. ...  4.  2.  2.]
 [ 2.  1.  4. ...  2.  4.  3.]
 [ 4.  2.  5. ... nan nan nan]
 ...
 [ 4.  4.  4. ...  1.  1.  1.]
 [ 4.  1.  2. ...  1.  1.  3.]
 [ 3.  5.  5. ...  5.  5.  2.]]


We need to know which indices have nans, and which do not.  Write a function which takes in a matrix like `foodMatrix` and returns a list of tuples containing the rows and columns of a rating.  That is, if your function is called on `foodMatrix`, it *should* contain `(0,0)` but should *not* contain `(2,17)`. This is because 0,0 is rated (with 4 stars) and 2,17 is not (it's a NaN).

You may use generative AI for this portion, and all portions until stated otherwise.

In [5]:
def findRatings(mat):
    indeces = []
    numRows, numCols = mat.shape
    for row in range(numRows):
        for col in range(numCols):
            if not np.isnan(mat[row][col]):
                indeces.append((row, col))
    return indeces

Run your function on `foodMatrix` and use sklearn's function `train_test_split()` to split the list into a training set of indices and a testing set.

In [6]:
totalIndeces = findRatings(foodMatrix)
train, test = train_test_split(totalIndeces, test_size=0.1)

We will need an error function so we can tell if our completion is working. We'll use root mean squared error (RMSE), which is defined as: 

$$\sqrt{\frac{\sum_{(i,j)\in T}(\text{original}[i,j]-\text{reproduction}[i,j])^2}{|T|}}$$

where $T$ is a set of indices, `original` is the original matrix, and `reproduction` is your value from the completed matrix.

Your function should take three arguments, the original matrix, the reproduction, and a set of indices (like your training or testing sets you created above).  Test your function with some quick sanity checks - if you call it with the same matrix for original and reproduction, do you get back 0?  If you very slightly change a value in the reproduction which appears in the index set, do you get back a non-zero but small value?

In [7]:
def rmse(org, rep, indeces):
    sum = 0
    for row, col in indeces:
        diff = org[row][col] - rep[row][col]
        sum += diff * diff
    return np.sqrt(sum/len(indeces))



# using same matrix, should get 0
print(f"Same matrix error: {rmse(foodMatrix, foodMatrix, train)}")

# using slightly altered matrix, should get very small error
newMatrix = deepcopy(foodMatrix)
newMatrix[0][0] = 1
print(f"Altered matrix error: {rmse(foodMatrix, newMatrix, train)}")

Same matrix error: 0.0
Altered matrix error: 0.03079394062236661


We need to make our initial guesses of $P$ and $Q$.  Write a function which takes in a matrix's size and assumed rank, and creates appropriate $P$ and $Q$.  A danger here is if you use large values, `P@Q` will overflow, so start with small, random values close to zero.

In [8]:
# returns (P, Q)
def createPQ(dimension, rank):
    return (np.random.rand(dimension[0], rank), np.random.rand(rank, dimension[1]))

P, Q = createPQ(foodMatrix.shape, 10)
print(P)
print("----------------")
print(Q)

[[0.76918713 0.54537891 0.62532785 ... 0.56925452 0.03413963 0.82197688]
 [0.46921526 0.7421613  0.11003365 ... 0.18552114 0.93533092 0.05919732]
 [0.71169163 0.45302281 0.91282262 ... 0.13517646 0.62027315 0.7116855 ]
 ...
 [0.12033096 0.33066243 0.20652676 ... 0.07875383 0.2722366  0.0247043 ]
 [0.1247356  0.91187243 0.51405045 ... 0.14043387 0.53256987 0.38386902]
 [0.07306509 0.80938827 0.94034749 ... 0.83856481 0.73500079 0.88167336]]
----------------
[[0.62403632 0.60300322 0.50677801 0.21829775 0.63004021 0.9859105
  0.69015227 0.57214428 0.64023838 0.89602977 0.21103068 0.66108914
  0.70805325 0.71856734 0.84535905 0.93321328 0.86201811 0.42354103]
 [0.72133255 0.04249176 0.14104644 0.4336681  0.89264981 0.91512324
  0.91521858 0.10451682 0.42578513 0.17202789 0.94802908 0.40228155
  0.80144581 0.03220412 0.39927492 0.90453767 0.41151031 0.61357729]
 [0.11716004 0.31196895 0.42097983 0.31720997 0.566128   0.3368419
  0.48742734 0.32644636 0.58793952 0.41373806 0.32447384 0.0528

It's business time.  Make a function called `epoch`, which runs one epoch of training.  It should accept your original matrix, your current values of $P$ and $Q$, a stepsize, a training set of indices and a testing set of indices.  It should train on all elements of the training set, then calculate the training and testing errors.  It should return the updated matrices $P$, $Q$, and the scalars of the training and testing errors.

You may **NOT** use generative AI on this portion.  You may not use functions from other libraries that purport to perform matrix completion.

In [ ]:
def epoch(original, P, Q, a, train, test):
    

Now you have everything you need.  Choose some assumed rank, stepsize, and number of epochs, and run matrix completion.  Keep lists of the training and testing errors after each epoch.  You may **NOT** use generative AI on this portion.

Plot your training and testing errors as a function of the epoch. You **may** use generative AI on this portion.

Play around with the stepsize, rank, and number of epochs.  Below, answer a couple questions, by running experiments and showing me error plots that support your argument.

- What happened when your stepsize was too small? Too large?
- Relate the value of rank to bias and variance.  What would cause high bias and low variance?  What about low bias and high variance?
- Is more epochs always better?

You may **NOT** use generative AI on this portion.